In [1]:
import sys  
import os
sys.path.insert(0, '..')
from AT_funs_appendix import *
from AT_funs import read_xlsx

In [2]:
filename_bm = 'FTSEMIB.MI.xlsx'
bm_df = read_xlsx(filename_bm)
bm_name = filename_bm.replace('xlsx', '')
bm_df['date'] = pd.to_datetime(bm_df['date'])
bm_df['USD'] = 1

In [3]:
params = ['2014-12-31', None, 63, 0.05, 0.05, 1.5, 2,5,2.5,3]
start,end,vlty_n,dist_pct,retrace_pct,threshold,dgt,d_vol,r_vol,lvl= [params[h] for h in range(len(params))]

rel_var = ['FTSEMIB.MI','close', 'USD']
bm_ticker, bm_col, ccy_col = [rel_var[h] for h in range(len(rel_var))]
# lvl

In [4]:
window = 100
st= fast = 50
lt = slow = 200

batch_size = 1
show_batch = True
save_ticker_df = False
save_last_row_df = False
save_regime_df = False

In [5]:
# website = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
web_df_cols = ['Symbol','Security','GICS Sector','GICS Sub-Industry']
regime_cols = ['rg','rrg',
    'smac'+str(st)+str(lt),'smar'+str(st)+str(lt), 'bohl'+str(slow),
    'borr'+str(slow),'tth'+str(fast)+'l'+str(slow),'ttr'+str(fast)+'r'+str(slow)]

swings_cols = ['flr_dt','flr','clg_dt', 'clg', 'rg_ch', 
    'hi'+str(lvl)+'_dt','hi'+str(lvl),'lo'+str(lvl)+'_dt','lo'+str(lvl) ,
      'rflr_dt', 'rflr', 'rclg_dt', 'rclg', 'rrg_ch',
    'rh'+str(lvl)+'_dt','rh'+str(lvl),'rl'+str(lvl)+'_dt','rl'+str(lvl) ]
symbol_cols = ['ticker','date','close']

last_row_df_cols = symbol_cols+['score']+regime_cols+swings_cols

In [6]:
folder_path = os.getcwd()  
  
# Get all xlsx files in folder (except ftsemib.xlsx)  
tickers_list = [file for file in os.listdir(folder_path) if file.endswith(".xlsx") and file != "FTSEMIB.MI.xlsx" and file != "sectors.xlsx"]  
tickers_list = tickers_list[:20] 
# tickers_list

In [7]:
# Read all xlsx files  
dfs = []
    
for file in tickers_list:      
    file_path = os.path.join(folder_path, file)      
    df = pd.read_excel(file_path)  
    if df.shape[0] >= 1000:  
        dfs.append(df)  

In [8]:
regime_df = pd.DataFrame()
last_row_df = pd.DataFrame()
signal_df = pd.DataFrame()
signal_list = []
last_row_list = []
failed = []

In [9]:
loop_size = len(tickers_list)+1

In [10]:
def last_row_dictionary(df, ticker):
    
    df_cols = list(df.columns)
    col_dict = {'Symbol':str.upper(ticker),'date':df['date'].max().strftime('%Y%m%d')}
    for i, col_name in enumerate(df_cols):
        if pd.isnull(df.iloc[-1,i]):
            try:
                last_index = df[pd.notnull(df.iloc[:,i])].index[-1]
                len_last_index = len(df[:last_index]) - 1
                col_dict.update({col_name + '_dt': last_index.strftime('%Y%m%d')})
                col_dict.update({col_name : df.iloc[len_last_index,i]})
            except:
                col_dict.update({col_name + '_dt':np.nan})
                col_dict.update({col_name : np.nan})
        else:
            col_dict.update({col_name : df.iloc[-1,i]})
    return col_dict

def find_last_change(df, regime_cols):
    date_cols = ['ticker','date']
    signal = df[date_cols + regime_cols] 
    signal = pd.melt(signal, id_vars=['ticker', 'date'], var_name='regime', value_name='signal') 
    signal_change = signal.groupby('regime').apply(lambda x: x[x['signal'].diff() != 0].tail(1)) 
    signal_change = signal_change.drop(columns=['regime']) 
    signal_change = signal_change.reset_index() 
    signal_change = signal_change.pivot(index='ticker', columns='regime', values='date') 
    return signal_change

In [11]:

for t in range(1,loop_size): 
    m = (t - 1) * batch_size
    n = t * batch_size
    batch_list = tickers_list[m:n]
    if show_batch:
        print(batch_list,m,n)
        
    try:
        batch_download = dfs[t-1]
        
        for flat, ticker in enumerate(batch_list):
            df = swings2(batch_download,lvl,vlty_n,dgt,d_vol,dist_pct,r_vol,retrace_pct,bm_df,bm_col,rel = False)
            df = regime(df,vlty_n,threshold,lvl=2,rel = False)
            df = swings2(batch_download,lvl,vlty_n,dgt,d_vol,dist_pct,r_vol,retrace_pct,bm_df,bm_col,rel = True)
            df = regime(df,vlty_n,threshold,lvl=2,rel = True)          
            _o,_h,_l,_c = lower_upper_OHLC(df,relative = False)
            df['sma'+str(_c)[:1]+str(st)+str(lt)] = regime_sma(df,_c,st,lt)
            df['bo'+str(_h)[:1]+str(_l)[:1]+ str(slow)] = regime_breakout(df,_h,_l,window)
            df['tt'+str(_h)[:1]+str(fast)+str(_l)[:1]+ str(slow)] = turtle_trader(df, _h, _l, slow, fast)
            _o,_h,_l,_c = lower_upper_OHLC(df,relative = True) 
            df['sma'+str(_c)[:1]+str(st)+str(lt)] = regime_sma(df,_c,st,lt)
            df['bo'+str(_h)[:1]+str(_l)[:1]+ str(slow)] = regime_breakout(df,_h,_l,window)
            df['tt'+str(_h)[:1]+str(fast)+str(_l)[:1]+ str(slow)] = turtle_trader(df, _h, _l, slow, fast)
            df_signal = find_last_change(df, regime_cols)
            signal_list.append(df_signal)
                
        try:
            last_row_list.append(last_row_dictionary(df, ticker))
        except:
            failed.append(ticker) 
            
        # try:
        #     signal_list.append(find_last_change(df, date_cols, regime_cols))
        # except:
        #     failed.append(ticker) 
            
    except:
        failed.append(ticker)
last_row_df = pd.DataFrame.from_dict(last_row_list)
signal_df = pd.concat(signal_list, axis=0, ignore_index=True) 

if save_last_row_df:
    last_row_df.to_csv('last_row_df_'+ str(last_row_df['date'].max())+'.csv', date_format='%Y%m%d')
print('failed',failed)

['RLT.MI.xlsx'] 0 1


/tmp/ipykernel_1242583/3854100217.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  signal_change = signal.groupby('regime').apply(lambda x: x[x['signal'].diff() != 0].tail(1))


['AIM.MI.xlsx'] 1 2


/tmp/ipykernel_1242583/3854100217.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  signal_change = signal.groupby('regime').apply(lambda x: x[x['signal'].diff() != 0].tail(1))


['ABP.MI.xlsx'] 2 3


...

/tmp/ipykernel_1242583/3854100217.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  signal_change = signal.groupby('regime').apply(lambda x: x[x['signal'].diff() != 0].tail(1))


['CAI.MI.xlsx'] 9 10


/tmp/ipykernel_1242583/3854100217.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  signal_change = signal.groupby('regime').apply(lambda x: x[x['signal'].diff() != 0].tail(1))
/home/laceto/AT/scouting/../AT_funs_appendix.py:351: RuntimeWarning: divide by zero encountered in scalar divide
  if (vlty > 0) & (retrace_vol > 0) & ((round(retracement / vlty ,1) - retrace_vol) > 0):


['GHC.MI.xlsx'] 10 11
['AEF.MI.xlsx'] 11 12
['ART.MI.xlsx'] 12 13
['TPRO.MI.xlsx'] 13 14
['ESPE.MI.xlsx'] 14 15
['EUK.MI.xlsx'] 15 16
['MTVFR.MI.xlsx'] 16 17
['CE.MI.xlsx'] 17 18
['FAE.MI.xlsx'] 18 19
['ACQ.MI.xlsx'] 19 20
failed ['GHC.MI.xlsx', 'AEF.MI.xlsx', 'AEF.MI.xlsx', 'AEF.MI.xlsx', 'AEF.MI.xlsx', 'AEF.MI.xlsx', 'AEF.MI.xlsx', 'AEF.MI.xlsx', 'AEF.MI.xlsx']


/tmp/ipykernel_1242583/3854100217.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  signal_change = signal.groupby('regime').apply(lambda x: x[x['signal'].diff() != 0].tail(1))


In [12]:
last_row_df['score']= last_row_df[regime_cols].sum(axis=1)
sectors = pd.read_excel('sectors.xlsx')

In [13]:
# batch_download = dfs[0]
# df = swings2(batch_download,lvl,vlty_n,dgt,d_vol,dist_pct,r_vol,retrace_pct,bm_df,bm_col,rel = False)
# df = regime(df,vlty_n,threshold,lvl=2,rel = False)
# df = swings2(batch_download,lvl,vlty_n,dgt,d_vol,dist_pct,r_vol,retrace_pct,bm_df,bm_col,rel = True)
# df = regime(df,vlty_n,threshold,lvl=2,rel = True)          
# _o,_h,_l,_c = lower_upper_OHLC(df,relative = False)
# df['sma'+str(_c)[:1]+str(st)+str(lt)] = regime_sma(df,_c,st,lt)
# df['bo'+str(_h)[:1]+str(_l)[:1]+ str(slow)] = regime_breakout(df,_h,_l,window)
# df['tt'+str(_h)[:1]+str(fast)+str(_l)[:1]+ str(slow)] = turtle_trader(df, _h, _l, slow, fast)
# _o,_h,_l,_c = lower_upper_OHLC(df,relative = True) 
# df['sma'+str(_c)[:1]+str(st)+str(lt)] = regime_sma(df,_c,st,lt)
# df['bo'+str(_h)[:1]+str(_l)[:1]+ str(slow)] = regime_breakout(df,_h,_l,window)
# df['tt'+str(_h)[:1]+str(fast)+str(_l)[:1]+ str(slow)] = turtle_trader(df, _h, _l, slow, fast)
# df.columns

In [14]:
regime_df = sectors.set_index('ticker').join(
    last_row_df[last_row_df_cols].set_index('ticker'), how='inner').sort_values(by='score')

if save_regime_df:
    regime_df.to_csv('regime_df_'+ str(last_row_df['date'].max())+'.csv', date_format='%Y%m%d')

In [15]:
groupby_cols = ['score'] + regime_cols
sort_key = ['name', 'sector']
regime_df.groupby(sort_key)[groupby_cols].mean().sort_values(
    by= 'score').style.background_gradient(
    subset= groupby_cols,cmap= 'RdYlGn').format('{:.1g}')

,,score,rg,rrg,smac50200,smar50200,bohl200,borr200,tth50l200,ttr50r200
name,sector,,,,,,,,,
AEFFE,Consumer Non-Durables,-8,-1,-1,-1,-1,-1,-1,-1,-1
RATTI,Consumer Non-Durables,-8,-1,-1,-1,-1,-1,-1,-1,-1
MAPS,Technology Services,-7,-1,-1,-1,-1,-1,-1,0,-1
RELATECH,Technology Services,-7,0,-1,-1,-1,-1,-1,-1,-1
EUKEDOS,Health Services,-6,1,-1,-1,-1,-1,-1,-1,-1
GEFRAN,Electronic Technology,-4,1,1,-1,-1,-1,-1,-1,-1
GAROFALO HEALTH CARE,Health Services,0,1,-1,1,-1,1,-1,1,-1
4AIM SICAF,Finance,3,0,-1,1,1,1,1,0,0
CREDEM,Finance,8,1,1,1,1,1,1,1,1


In [16]:
groupby_cols = ['score'] + regime_cols
sort_key = ['sector']
regime_df.groupby(sort_key)[groupby_cols].mean().sort_values(
    by= 'score').style.background_gradient(
    subset= groupby_cols,cmap= 'RdYlGn').format('{:.1g}')

pd.set_option('display.float_format', lambda x: '%.2f' % x)
groupby_cols = ['score'] + regime_cols
sort_key = ['sector']
regime_df.groupby(sort_key)[groupby_cols].sum().sort_values(
    by= 'score').style.background_gradient(
    subset= groupby_cols,cmap= 'RdYlGn').format('{:.1g}')

,score,rg,rrg,smac50200,smar50200,bohl200,borr200,tth50l200,ttr50r200
sector,,,,,,,,,
Consumer Non-Durables,-2e+01,-2,-2,-2,-2,-2,-2,-2,-2
Technology Services,-1e+01,-1,-2,-2,-2,-2,-2,-1,-2
Health Services,-6,2,-2,0,-2,0,-2,0,-2
Electronic Technology,4,2,2,0,0,0,0,0,0
Consumer Services,8,1,1,1,1,1,1,1,1
Finance,1e+01,1,0,2,2,2,2,1,1


In [17]:
# groupby_cols = ['score'] + regime_cols
# sort_key = ['GICS Sub-Industry']
# regime_df.groupby(sort_key)[groupby_cols].mean().sort_values(
#     by= 'score').style.background_gradient(
#     subset= groupby_cols,cmap= 'RdYlGn').format('{:.1g}')

In [18]:
# groupby_cols = ['score'] + regime_cols
# sort_key = ['GICS Sector','GICS Sub-Industry']
# regime_df.groupby(sort_key)[groupby_cols].mean().sort_values(
#     by= ['GICS Sector','score']).style.background_gradient(
#     subset= groupby_cols,cmap= 'RdYlGn').format('{:.1g}')

In [43]:
signal_df = pd.concat(signal_list, axis=0, ignore_index=False) 
signal_df

regime,bohl200,borr200,rg,rrg,smac50200,smar50200,tth50l200,ttr50r200
ticker,,,,,,,,
RLT.MI,2024-04-15 07:00:00,2024-02-29 08:00:00,2024-05-10 15:35:08,2024-02-19 08:00:00,2024-04-18 07:00:00,2022-11-22 08:00:00,2024-03-07 08:00:00,2024-02-29 08:00:00
AIM.MI,2023-12-18 08:00:00,2023-12-22 08:00:00,2024-04-04 07:00:00,2024-04-12 07:00:00,2023-12-22 08:00:00,2024-01-16 08:00:00,2024-04-30 07:00:00,2024-04-30 07:00:00
MAPS.MI,2024-04-11 07:00:00,2023-03-28 07:00:00,2024-04-03 07:00:00,2024-03-04 08:00:00,2024-04-16 07:00:00,2023-01-05 08:00:00,2024-05-09 07:00:00,2024-02-26 08:00:00
GE.MI,2024-05-09 07:00:00,2023-05-17 07:00:00,2024-05-03 07:00:00,2024-05-03 07:00:00,2023-07-10 07:00:00,2022-10-26 07:00:00,2024-05-09 07:00:00,2024-05-08 07:00:00
AVIO.MI,2024-02-28 08:00:00,2024-02-29 08:00:00,2024-02-01 08:00:00,2024-01-31 08:00:00,2024-02-29 08:00:00,2024-04-22 07:00:00,2024-03-14 08:00:00,2024-04-18 07:00:00
RAT.MI,2024-01-19 08:00:00,2022-08-16 07:00:00,2023-12-08 08:00:00,2023-11-02 08:00:00,2022-06-21 07:00:00,2022-10-05 07:00:00,2024-01-16 08:00:00,2022-11-30 08:00:00
CAI.MI,2023-12-20 08:00:00,2024-04-09 07:00:00,2023-11-02 08:00:00,2024-04-23 07:00:00,2024-01-05 08:00:00,2024-04-25 07:00:00,2023-11-02 08:00:00,2024-04-09 07:00:00
GHC.MI,2023-05-05 07:00:00,2024-03-12 08:00:00,2024-01-15 08:00:00,2024-04-26 07:00:00,2023-06-01 07:00:00,2024-03-07 08:00:00,2023-05-16 07:00:00,2024-03-18 08:00:00
AEF.MI,2022-03-03 08:00:00,2022-03-07 08:00:00,2024-04-15 07:00:00,2022-03-01 08:00:00,2022-04-05 07:00:00,2022-04-13 07:00:00,2024-03-19 08:00:00,2024-03-08 08:00:00
